In [31]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

# Data beans common themes using RAG (retrieval augmented generation)

This notebook shows how to implement a RAG procedure inside BigQuery to extract common review themes by product name, it performs the following steps:
- Create LLM and embeddings models (`gemini-pro` and `gecko`)
- Embbed the `review_text` column of the `customer_review` table
- Create an index on the embeddings for faster retrieval
- Wraps inside a BigQuery procedure a RAG implementation

In [32]:
PROJECT_ID = "bt-int-wod-masterclass-c3f3"
REGION = "EU"
DATASET_ID = "data_beans_curated"
CONNECTION_NAME = "vertex-ai"

In [33]:
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID, location=REGION)

In [34]:
client.query(f"CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}_local` OPTIONS(location = 'EU');").result()

client.query(
   f"""CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET_ID}_local.emb_model`
      REMOTE WITH CONNECTION `{REGION}.{CONNECTION_NAME}`
      OPTIONS(ENDPOINT = 'textembedding-gecko@latest');"""
 ).result()

client.query(
    f"""CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET_ID}_local.llm_model`
       REMOTE WITH CONNECTION `{REGION}.{CONNECTION_NAME}`
       OPTIONS(ENDPOINT = 'gemini-pro');
""").result()

In [ ]:
client.query(
    f"""select review_text as content from `{PROJECT_ID}.{DATASET_ID}.customer_review`"""
).result().to_dataframe()

Create table with embeddings

In [ ]:
client.query(
    f"""CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}_local.customer_review_embedded`
        as
        SELECT *
        FROM
          ML.GENERATE_TEXT_EMBEDDING(
            MODEL `{PROJECT_ID}.{DATASET_ID}_local.emb_model`,
            (select review_text as content from `{PROJECT_ID}.{DATASET_ID}.customer_review`)
        );"""
).result()

In [ ]:
client.query(
    f"""CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}_local.customer_review_embedded_laurenz`
        as
        SELECT row_number() over () as rowid
             , cre.*
        FROM `{PROJECT_ID}.{DATASET_ID}_local.customer_review_embedded` cre
        WHERE ml_embed_text_status = ""; -- use only rows without errors
        """
).result()

In [36]:
client.query(
    f"""select * from `{PROJECT_ID}.{DATASET_ID}_local.customer_review_embedded`"""
).result().to_dataframe()

,text_embedding,statistics,ml_embed_text_status,content
0,"[-0.005726990289986134, -0.056526798754930496,...","{""token_count"":1,""truncated"":false}",,good
1,"[0.0074938093312084675, -0.071085624396801, -0...","{""token_count"":1,""truncated"":false}",,Good
2,"[-0.005726990289986134, -0.056526798754930496,...","{""token_count"":1,""truncated"":false}",,good
3,"[-0.008162016980350018, -0.03362084552645683, ...","{""token_count"":1,""truncated"":false}",,Nice
4,"[0.005998166743665934, -0.051547858864068985, ...","{""token_count"":1,""truncated"":false}",,Great
...,...,...,...,...
7999,"[0.04008382931351662, -0.01864839531481266, -0...","{""token_count"":37,""truncated"":false}",,I had a great experience at this coffee truck....
8000,"[0.04008382931351662, -0.01864839531481266, -0...","{""token_count"":37,""truncated"":false}",,I had a great experience at this coffee truck....
8001,"[0.04008382931351662, -0.01864839531481266, -0...","{""token_count"":37,""truncated"":false}",,I had a great experience at this coffee truck....
8002,"[0.04008382931351662, -0.01864839531481266, -0...","{""token_count"":37,""truncated"":false}",,I had a great experience at this coffee truck....


Create vector index on the embeddings table

In [ ]:
client.query(
  f"""CREATE OR REPLACE VECTOR INDEX `{PROJECT_ID}.{DATASET_ID}_local.reviews_index`
  ON `{PROJECT_ID}.{DATASET_ID}_local.customer_review_embedded_laurenz`(text_embedding)
  OPTIONS(distance_type='COSINE', index_type='IVF');"""
).result()

Create a procedure to implement a RAG pipeline

In [45]:
client.query(
    f"""
    CREATE OR REPLACE PROCEDURE {DATASET_ID}_local.common_themes_by_menu_items(menu_item STRING, OUT themes STRING)
    BEGIN
    WITH embedding as (
      SELECT text_embedding,
             content AS query
      FROM ML.GENERATE_TEXT_EMBEDDING( MODEL `{PROJECT_ID}.{DATASET_ID}_local.emb_model`,
          (
          SELECT
            CAST(menu_item AS STRING) AS content))
    ),

    neighbors as (
      SELECT *
      FROM VECTOR_SEARCH( TABLE `{PROJECT_ID}.{DATASET_ID}_local.customer_review_embedded_laurenz`,
        'text_embedding',
        (SELECT * FROM embedding),
        top_k => 50,
        distance_type => 'COSINE'
      )
    )
    SELECT
      ml_generate_text_llm_result AS generated
    FROM
      ML.GENERATE_TEXT(
        MODEL `{PROJECT_ID}.{DATASET_ID}_local.llm_model`,
        (
        SELECT
          CONCAT(
            'Extract common themes from the following reviews: ',
            STRING_AGG(base.content, ';'),
            '. Reply in JSON format with the keys: item_name, themes'
          ) AS prompt
        FROM neighbors
        ),
        STRUCT(
          0.4 AS temperature,
          300 AS max_output_tokens,
          0.5 AS top_p,
          5 AS top_k,
          TRUE AS flatten_json_output
        )
      );
    END;
"""
).result()

Executes the procedure

In [46]:
out = client.query(f"""
  DECLARE themes STRING;
  CALL {DATASET_ID}_local.common_themes_by_menu_items('location', themes);
"""
).result()

i = 0
for row in next(out):
    print(row)
    i += 1
    if i > 10:
        break

```json
{
  "item_name": "Location",
  "themes": [
    "Good location",
    "Dirty location",
    "Noisy location",
    "Overcrowded location",
    "Quiet location",
    "Convenient location",
    "Cozy location",
    "Clean location",
    "Bad location"
  ]
}
```

**Explanation of Themes:**

* **Good location:** This theme is mentioned in several positive reviews, highlighting the convenience and accessibility of the location.
* **Dirty location:** This theme is a recurring negative aspect, indicating that cleanliness is a major concern for some customers.
* **Noisy location:** Several reviews mention the noise level as a negative factor, making it difficult to relax or enjoy the experience.
* **Overcrowded location:** Some customers found the location to be too crowded, leading to a lack of seating and a less enjoyable experience.
* **Quiet location:** In contrast to the noisy locations, some customers appreciated the peacefulness and tranquility of certain locations.
* **Convenient 